In [3]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 116.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 121.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 9.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 99.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install qiskit_algorithms


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.primitives import Sampler
from qiskit_algorithms import IterativeAmplitudeEstimation, EstimationProblem


def f(x):
    return x**3 - 5*x + 1  


def f_prime_analytical(x):
    return 3*x**2 - 5


def quantum_derivative_estimation(x):
    
    scale_factor = 1.0
    scaled_x = x
    
    if abs(x) > 1:
        scale_factor = abs(x) + 0.1 
        scaled_x = x / scale_factor
        print(f"Scaling x={x} to scaled_x={scaled_x} with factor {scale_factor}")
    
  
    delta = 0.01 / scale_factor  
    x_plus = min(scaled_x + delta, 1)
    x_minus = max(scaled_x - delta, -1)
    
    
    qc_plus = QuantumCircuit(3)
    qc_plus.h(0)  
    qc_plus.ry(2 * np.arcsin(x_plus), 1)  
    
    qc_minus = QuantumCircuit(3)
    qc_minus.h(0) 
    qc_minus.ry(2 * np.arcsin(x_minus), 1) 
   
    combined_qc = QuantumCircuit(3)
    combined_qc.h(0)
    combined_qc.append(qc_plus.to_instruction(), [0, 1, 2])
    combined_qc.x(0)
    combined_qc.append(qc_minus.to_instruction(), [0, 1, 2])
    combined_qc.x(0)
    
   
    problem = EstimationProblem(
        state_preparation=combined_qc,
        objective_qubits=[1]
    )
    
    
    iae = IterativeAmplitudeEstimation(
        epsilon_target=0.1, 
        alpha=0.2,           
        sampler=Sampler()    
    )
    
    
    try:
        result = iae.estimate(problem)
        if result.estimation is None:
            print("QAE returned None, using analytical derivative as fallback.")
            return f_prime_analytical(x)
        
      
        estimated_derivative = (result.estimation - 0.5) * 20  
        
       
        estimated_derivative = estimated_derivative * scale_factor
        
      
        alpha = 0.7  
        blended_derivative = alpha * estimated_derivative + (1-alpha) * f_prime_analytical(x)
        
        return blended_derivative
        
    except Exception as e:
        print(f"QAE failed: {e}")
       
        return f_prime_analytical(x)


def newton_raphson_qae(x0, tol=1e-6, max_iter=20):
    x = x0
    history = [x0]
    
    for i in range(max_iter):
        fx = f(x)
        
        
        f_prime_x = quantum_derivative_estimation(x)
        
        
        if abs(f_prime_x) < tol:
            print(f"Iteration {i+1}: Derivative too small ({f_prime_x}), using regularization.")
            f_prime_x = np.sign(f_prime_x) * max(abs(f_prime_x), 0.1)  # Regularize
        
       
        step = fx / f_prime_x
        
     
        if abs(step) > 1.0:
            damping = 1.0 / abs(step)
            step *= damping
            print(f"Damping applied: {damping:.4f}")
        
        x_new = x - step
        
      
        print(f"Iteration {i+1}: x = {x_new:.6f}, f(x) = {fx:.6f}, f'(x) = {f_prime_x:.6f}")
        
       
        if abs(x_new - x) < tol or abs(fx) < tol:
            print(f"Converged after {i+1} iterations.")
            return x_new
        
        
        history.append(x_new)
        if i >= 3 and is_oscillating(history[-4:]):
            print("Oscillation detected, applying stabilization.")
            x_new = (history[-1] + history[-2] + history[-3]) / 3  # Average last 3 points
        
        x = x_new
    
    print(f"Maximum iterations ({max_iter}) reached without convergence.")
    return x


def is_oscillating(points):
    if len(points) < 4:
        return False
   
    diff1 = points[1] - points[0]
    diff2 = points[2] - points[1]
    diff3 = points[3] - points[2]
    
    return (diff1 * diff2 < 0) and (diff2 * diff3 < 0)


initial_guess = 0.5  
root = newton_raphson_qae(initial_guess)
print(f"Estimated root: {root}")
print(f"Function value at estimated root: {f(root)}")


def newton_raphson_analytical(x0, tol=1e-6, max_iter=20):
    x = x0
    for i in range(max_iter):
        fx = f(x)
        f_prime_x = f_prime_analytical(x)
        if abs(f_prime_x) < tol:
            break
        x_new = x - fx / f_prime_x
        if abs(x_new - x) < tol:
            return x_new
        x = x_new
    return x

analytical_root = newton_raphson_analytical(initial_guess)
print(f"\nAnalytical Newton-Raphson root: {analytical_root}")
print(f"Function value at analytical root: {f(analytical_root)}")

/tmp/ipykernel_218/1346105656.py:59: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler=Sampler()    # Explicit sampler


Iteration 1: x = 1.117718, f(x) = -1.375000, f'(x) = 2.225933
Scaling x=1.1177183814719855 to scaled_x=0.9178792062914256 with factor 1.2177183814719856
Damping applied: 0.0414
Iteration 2: x = 2.117718, f(x) = -3.192233, f'(x) = 0.132008
Scaling x=2.1177183814719855 to scaled_x=0.9549086120061708 with factor 2.2177183814719856
Iteration 3: x = 2.087590, f(x) = -0.091194, f'(x) = -3.026876
Scaling x=2.087590182811796 to scaled_x=0.9542875988447406 with factor 2.187590182811796
Iteration 4: x = 1.972126, f(x) = -0.340164, f'(x) = -2.946071
Scaling x=1.9721264058956272 to scaled_x=0.9517403958969494 with factor 2.0721264058956272
Iteration 5: x = 1.518684, f(x) = -1.190475, f'(x) = -2.625415
Scaling x=1.518683707648956 to scaled_x=0.938221408217394 with factor 1.6186837076489562
Damping applied: 0.3947
Iteration 6: x = 0.518684, f(x) = -3.090726, f'(x) = -1.220056
Iteration 7: x = 1.046270, f(x) = -1.453876, f'(x) = 2.755714
Scaling x=1.0462695326329103 to scaled_x=0.9127604833303854 wit